# EMA-Strategy Interactive Brokers execution

In [1]:
import ibapi
import pandas as pd

In [2]:
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)

C:\ProgramData\Anaconda3\python.exe
3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)]
sys.version_info(major=3, minor=8, micro=5, releaselevel='final', serial=0)


In [13]:
from ibapi.ticktype import TickTypeEnum

for i in range(91):
    print(TickTypeEnum.to_str(i), i)

BID_SIZE 0
BID 1
ASK 2
ASK_SIZE 3
LAST 4
LAST_SIZE 5
HIGH 6
LOW 7
VOLUME 8
CLOSE 9
BID_OPTION_COMPUTATION 10
ASK_OPTION_COMPUTATION 11
LAST_OPTION_COMPUTATION 12
MODEL_OPTION 13
OPEN 14
LOW_13_WEEK 15
HIGH_13_WEEK 16
LOW_26_WEEK 17
HIGH_26_WEEK 18
LOW_52_WEEK 19
HIGH_52_WEEK 20
AVG_VOLUME 21
OPEN_INTEREST 22
OPTION_HISTORICAL_VOL 23
OPTION_IMPLIED_VOL 24
OPTION_BID_EXCH 25
OPTION_ASK_EXCH 26
OPTION_CALL_OPEN_INTEREST 27
OPTION_PUT_OPEN_INTEREST 28
OPTION_CALL_VOLUME 29
OPTION_PUT_VOLUME 30
INDEX_FUTURE_PREMIUM 31
BID_EXCH 32
ASK_EXCH 33
AUCTION_VOLUME 34
AUCTION_PRICE 35
AUCTION_IMBALANCE 36
MARK_PRICE 37
BID_EFP_COMPUTATION 38
ASK_EFP_COMPUTATION 39
LAST_EFP_COMPUTATION 40
OPEN_EFP_COMPUTATION 41
HIGH_EFP_COMPUTATION 42
LOW_EFP_COMPUTATION 43
CLOSE_EFP_COMPUTATION 44
LAST_TIMESTAMP 45
SHORTABLE 46
FUNDAMENTAL_RATIOS 47
RT_VOLUME 48
HALTED 49
BID_YIELD 50
ASK_YIELD 51
LAST_YIELD 52
CUST_OPTION_COMPUTATION 53
TRADE_COUNT 54
TRADE_RATE 55
VOLUME_RATE 56
LAST_RTH_TRADE 57
RT_HISTORICAL_VO

In [1]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order import *

import threading
import time

class IBapi(EWrapper, EClient):
	
	def __init__(self):
		EClient.__init__(self, self)
	
	def nextValidId(self, orderId: int):
		super().nextValidId(orderId)
		self.nextorderId = orderId
		print('The next valid order id is: ', self.nextorderId)

	def orderStatus(self, orderId, status, filled, remaining, avgFullPrice, permId, parentId, lastFillPrice, clientId, whyHeld, mktCapPrice):
		print('orderStatus - orderid:', orderId, 'status:', status, 'filled', filled, 'remaining', remaining, 'lastFillPrice', lastFillPrice)
	
	def openOrder(self, orderId, contract, order, orderState):
		print('openOrder id:', orderId, contract.symbol, contract.secType, '@', contract.exchange, ':', order.action, order.orderType, order.totalQuantity, orderState.status)

	def execDetails(self, reqId, contract, execution):
		print('Order Executed: ', reqId, contract.symbol, contract.secType, contract.currency, execution.execId, execution.orderId, execution.shares, execution.lastLiquidity)


def run_loop():
	app.run()

def FX_order(symbol):
	contract = Contract()
	contract.symbol = symbol[:3]
	contract.secType = 'CASH'
	contract.exchange = 'IDEALPRO'
	contract.currency = symbol[3:]
	return contract

app = IBapi()
app.connect('127.0.0.1', 7497, 123)

app.nextorderId = None

#Start the socket in a thread
api_thread = threading.Thread(target=run_loop, daemon=True)
api_thread.start()

#Check if the API is connected via orderid
while True:
	if isinstance(app.nextorderId, int):
		print('connected')
		print()
		break
	else:
		print('waiting for connection')
		time.sleep(1)

#Create order object
app.nextorderId += 1
order = Order()
order.action = 'BUY'
order.totalQuantity = 100000
order.orderType = 'LMT'
order.lmtPrice = '1.10'
order.orderId = app.nextorderId
app.nextorderId += 1
order.transmit = False

#Create stop loss order object
stop_order = Order()
stop_order.action = 'SELL'
stop_order.totalQuantity = 100000
stop_order.orderType = 'STP'
stop_order.auxPrice = '0.96'
stop_order.orderId = app.nextorderId
app.nextorderId += 1
stop_order.parentId = order.orderId
stop_order.transmit = True

#Place orders
time.sleep(1)
app.placeOrder(order.orderId, FX_order('EURUSD'), order)
time.sleep(1)
app.placeOrder(stop_order.orderId, FX_order('EURUSD'), stop_order)

app.disconnect()

ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:cashhmds
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefil


waiting for connection
The next valid order id is:  52
connected



ERROR 53 2168 Warning: The 'EtradeOnly' order attribute is not supported.
ERROR 53 2169 Warning: The 'FirmQuoteOnly' order attribute is not supported.


In [1]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract

import threading
import time


class IBapi(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
    def tickPrice(self, reqId, tickType, price, attrib):
        if tickType == 9 and reqId == 1:
            print('The current ask price is: ', price)
            
    #Create contract object
    def Futures_contract(self, symbol, secType='FUT', exchange='GLOBEX', currency='USD'):
        ''' custom function to create contract '''
        contract = Contract()
        contract.symbol = symbol
        contract.secType = secType
        contract.exchange = exchange
        contract.currency = currency
        return contract

def run_loop():
    app.run()

app = IBapi()
app.connect('127.0.0.1', 7497, 123)

#Start the socket in a thread
api_thread = threading.Thread(target=run_loop, daemon=True)
api_thread.start()

time.sleep(1) #Sleep interval to allow time for connection to server



NQ_contract = app.Futures_contract('NQ')

#Request Market Data
app.reqMktData(1, NQ_contract, 9, False, False, [])

time.sleep(10) #Sleep interval to allow time for incoming price data
app.disconnect()

ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:euhmds
ERROR -1 2106 HMDS data farm connection is OK:fundfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefil
ERROR 1 321 Error validating request.-'bW' : cause - Please enter a local symbol or an expiry


In [27]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order import *

import threading
import time

class IBapi(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)

    def nextValidId(self, orderId: int):
        super().nextValidId(orderId)
        self.nextorderId = orderId
        print('The next valid order id is: ', self.nextorderId)
        
    def error(self, reqId, errorCode, errorString):
        if errorCode == 202:
            print('order canceled') 

    def orderStatus(self, orderId, status, filled, remaining, avgFullPrice, permId, parentId, lastFillPrice, clientId, whyHeld, mktCapPrice):
        print('orderStatus - orderid:', orderId, 'status:', status, 'filled', filled, 'remaining', remaining, 'lastFillPrice', lastFillPrice)

    def openOrder(self, orderId, contract, order, orderState):
        print('openOrder id:', orderId, contract.symbol, contract.secType, '@', contract.exchange, ':', order.action, order.orderType, order.totalQuantity, orderState.status)

    def execDetails(self, reqId, contract, execution):
        print('Order Executed: ', reqId, contract.symbol, contract.secType, contract.currency, execution.execId, execution.orderId, execution.shares, execution.lastLiquidity)


def run_loop():
    app.run()

#Function to create FX Order contract
def FX_order(symbol):
    contract = Contract()
    contract.symbol = symbol[:3]
    contract.secType = 'CASH'
    contract.exchange = 'IDEALPRO'
    contract.currency = symbol[3:]
    return contract

def STK_order_buy(symbol, quantity):
    contract = Contract()
    contract.symbol = symbol
    contract.secType = 'STK'
    contract.exchange = 'SMART'
    contract.currency = 'USD'
    
    #Create stop loss order object
    stop_order = Order()
    stop_order.action = 'SELL'
    stop_order.totalQuantity = quantity
    stop_order.orderType = 'STP'
    stop_order.auxPrice = '1.09'
    stop_order.orderId = app.nextorderId
    app.nextorderId += 1
    stop_order.parentId = order.orderId
    order.transmit = True
    
    #Create stop loss order object
    takeprofit_order = Order()
    takeprofit_order.action = 'SELL'
    takeprofit_order.totalQuantity = quantity
    takeprofit_order.orderType = 'STP'
    stop_order.auxPrice = '1.09'
    stop_order.orderId = app.nextorderId
    app.nextorderId += 1
    stop_order.parentId = order.orderId
    order.transmit = True
    
def STK_order_sell(symbol, quantity):
    contract = Contract()
    contract.symbol = symbol
    contract.secType = 'STK'
    contract.exchange = 'SMART'
    contract.currency = 'USD'
    
    #Create stop loss order object
    stop_order = Order()
    stop_order.action = 'BUY'
    stop_order.totalQuantity = quantity
    stop_order.orderType = 'STP'
    stop_order.auxPrice = '1.09'
    stop_order.orderId = app.nextorderId
    app.nextorderId += 1
    stop_order.parentId = order.orderId
    order.transmit = True
    
    #Create stop loss order object
    takeprofit_order = Order()
    takeprofit_order.action = 'BUY'
    takeprofit_order.totalQuantity = quantity
    takeprofit_order.orderType = 'STP'
    stop_order.auxPrice = '1.09'
    stop_order.orderId = app.nextorderId
    app.nextorderId += 1
    stop_order.parentId = order.orderId
    order.transmit = True
    

app = IBapi()
app.connect('127.0.0.1', 7497, 123)

app.nextorderId = None

#Start the socket in a thread
api_thread = threading.Thread(target=run_loop, daemon=True)
api_thread.start()

#Check if the API is connected via orderid
while True:
    if isinstance(app.nextorderId, int):
        print('connected')
        break
    else:
        print('waiting for connection')
        time.sleep(1)

#Create order object
enter_signal = 'BUY'

order = Order()
order.action = enter_signal
order.totalQuantity = 10
order.orderType = 'LMT'
order.lmtPrice = '160'

#Place order

# if order.action = 'BUY'
#     app.placeOrder(app.nextorderId, STK_order_buy('AAPL', quantity=order.totalQuantity), order)
    
# if order.action = 'SELL':
#     app.placeOrder(app.nextorderId, STK_order('AAPL', quantity=order.totalQuantity), order)
    
#app.nextorderId += 1

time.sleep(3)

#Cancel order 
print('cancelling order')
app.cancelOrder(app.nextorderId)

time.sleep(3)
app.disconnect()

waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for connection
waiting for

KeyboardInterrupt: 

In [3]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper

help(EClient)

Help on class EClient in module ibapi.client:

class EClient(builtins.object)
 |  EClient(wrapper)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, wrapper)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  calculateImpliedVolatility(self, reqId: int, contract: ibapi.contract.Contract, optionPrice: float, underPrice: float, implVolOptions: list)
 |      Call this function to calculate volatility for a supplied
 |      option price and underlying price. Result will be delivered
 |      via EWrapper.tickOptionComputation()
 |      
 |      reqId:TickerId -  The request id.
 |      contract:Contract -  Describes the contract.
 |      optionPrice:double - The price of the option.
 |      underPrice:double - Price of the underlying.
 |  
 |  calculateOptionPrice(self, reqId: int, contract: ibapi.contract.Contract, volatility: float, underPrice: float, optPrcOptions: list)
 |      Call this function to calculate option price and greek values
 |      for a 